# 3. Progress on Final Project (Reddit)

1. What is your proposed unit of analysis? In other words, if/when you end up building something like a spreadsheet, what are rows going to represent?
> I'd like to create visualizations for the composition of comments over time. For this, I think it makes most sense to have each row be the _content of a comment_, with an associated timestamp, associated post, and associated nest level. 

2. What specific measures associated with each unit do you want to collect? In other words, what are the columns in the spreadsheet going to be?
>(row) comment, (column) timestamp, (column) nest_level* (I'm getting write-file errors because not every item has nest_level. Will look into how to make exceptions)

3. Tell us what you've learned about the API:
* Are you going to be able to get the data you want with one API call or many? If more than one, how many?
* If it's more than one call, how will you know when you have collected all your data?
>It will be several calls. I also want to look at post titles, which is a separate endpoint from comments. I will need a way to associate these two datasets, so I'll need to find a common key among them. I think permalink might be a way to do this, but I will need to dig in further. How will I know if I've collected all my data? I think I'll try to break this up into notebooks and .tsv files that allow me to manually spot-check. Not confident enough in my skills to know the data is right the first time :) 

4. Make one API call and save the output to your desk in either a .json or .jsonl file. Be sure to share the code you used to do this. Be sure not to include any API keys in your notebook! **(See code below)

5. How big is the JSON file that you saved on your disk (i.e., in bytes or kilobytes)? If it is not your full dataset, what is your estimate for how much larger the full dataset will be? How big will the total dataset be? Is that a problem? 
>I will likely restrict my data source to the first two years of the subreddit's existence because this was the most volatile period of time and won't be too unwieldy a size to work with. If i change the dates so that I'm pulling one days' worth of data, I see over 1000 new, unique comments. This results in a file of 197KB. 197KB*(365*2) = 143 MB. I need to think about how I'm going to retreive more comments, but I think this is an acceptable limitation for now. 

In [22]:
import requests
import csv
import json

# Define the Pushshift API endpoint and subreddit
url = 'https://api.pushshift.io/reddit/comment/search'
subreddit = 'covidvaccinated'

# Define the start and end timestamps for the query
start_timestamp = 1640995200  # April 1, 2022 00:00:00 UTC
end_timestamp = 1651449599  # April 30, 2022 23:59:59 UTC

# Define the query parameters
params = {
    'subreddit': subreddit,
    'size': 1000,  # Number of comments to retrieve per request
    'after': start_timestamp,
    'before': end_timestamp,
}

response = requests.get(url, params=params)
data = response.json()



In [23]:
print (data)

{'data': [{'all_awardings': [], 'archived': False, 'associated_award': None, 'author': '[deleted]', 'author_created_utc': None, 'author_flair_background_color': '', 'author_flair_css_class': None, 'author_flair_template_id': None, 'author_flair_text': None, 'author_flair_text_color': 'dark', 'body': '[removed]', 'can_gild': True, 'collapsed': True, 'collapsed_because_crowd_control': None, 'collapsed_reason': None, 'collapsed_reason_code': 'DELETED', 'comment_type': None, 'controversiality': 0, 'created_utc': 1651446010, 'distinguished': None, 'edited': False, 'gilded': 0, 'gildings': {}, 'id': 'i6yniru', 'is_submitter': False, 'link_id': 't3_ude859', 'locked': False, 'no_follow': True, 'parent_id': None, 'permalink': '/r/CovidVaccinated/comments/ude859/when_should_a_12_year_old_get_a_booster_shot/i6yniru/', 'score': 1, 'score_hidden': False, 'send_replies': True, 'stickied': False, 'subreddit': 'CovidVaccinated', 'subreddit_id': 't5_3gppqs', 'subreddit_name_prefixed': 'r/CovidVaccinate

In [24]:
with open('covidvaccinated_april_2022.tsv', mode='w', newline='') as tsv_file:
    writer = csv.writer(tsv_file, delimiter='\t')
    writer.writerow(['created_utc', 'comment','nest_level'])

    # Write each comment to the .tsv file
    for comment in data["data"]:
        writer.writerow([comment['created_utc'], comment['body']])

In [ ]:
# ^^ I'm getting key errors on things that aren't applicable to all comments, such as nest_level. not all comments 
# will be nested. this is causing my write code to break. wondering if we can go over this in class. 